In [1]:
import sys
sys.path.append('/home/ez-flow/EY/python')
import bigquery_etl as bq
import pandas as pd
# pip install anytree
from anytree import Node, RenderTree,findall

In [5]:
import pytz

pytz.timezone('Asia/Seoul')

<DstTzInfo 'Asia/Seoul' LMT+8:28:00 STD>

# ---------------------
# Get Review Keywords Synonums

In [2]:
# get data from table
sql = '''
        SELECT distinct T1.*
        FROM (SELECT a.cmpl_fc1,a.cmpl_fc1_cd,b.cmpl_fc2,c.synonym,c.lemma
              FROM market-analysis-project-91130.taxonomy.cpl_fc1 a 
              FULL OUTER JOIN market-analysis-project-91130.taxonomy.cpl_fc2 b   on lower(a.cmpl_fc1_cd) = lower(b.cmpl_fc1_cd)
              FULL OUTER JOIN market-analysis-project-91130.taxonomy.rvw_kyw_syn c on lower(b.cmpl_fc2) = lower(c.cmpl_fc2) 
        ) T1
'''

ori_sql = '''
        SELECT distinct T1.*
        FROM (SELECT a.cmpl_fc1,a.cmpl_fc1_cd,b.cmpl_fc2
              FROM market-analysis-project-91130.taxonomy.org_cpl_fc1 a 
              FULL OUTER JOIN market-analysis-project-91130.taxonomy.org_cpl_fc2 b   on lower(a.cmpl_fc1_cd) = lower(b.cmpl_fc1_cd)
        ) T1
'''

df = bq.select_query(sql)
org_df = bq.select_query(ori_sql)

,cmpl_fc2,synonym,lemma


In [3]:
# Set Root
root = Node('Zinus Complain Factor')

In [4]:
# Add Nodes
for idx1, fc1 in enumerate( df['cmpl_fc1'].drop_duplicates() ) :
    # Complain Factor 1 Level
    fc1_node = Node(fc1, parent=root)
    for idx2, fc2 in enumerate( set(df[df['cmpl_fc1']==fc1]['cmpl_fc2']) ) :
        # Complain Factor 2 Level
        fc2_node = Node(fc2, parent=fc1_node)
        for idx3, s in enumerate( set(df[(df['cmpl_fc1']==fc1) & (df['cmpl_fc2']==fc2)]['synonym']) ) : 
            # Synonyms of Each Complain Factor 2 Words
            if s is None:
                pass
            else:
                s_node = Node(s, parent=fc2_node)
                
                

In [5]:
# Rendering with leaves count
for pre, fill, node in RenderTree(root):
    print("%s %s %d" % (pre, node.name, len(node.leaves)))
    
node.name

 Zinus Complain Factor 1107
├──  Size issue 93
│   ├──  low 6
│   │   ├──  too low 1
│   │   ├──  so low 1
│   │   ├──  low toppers 1
│   │   ├──  low mattress 1
│   │   ├──  low bed 1
│   │   └──  low topper 1
│   ├──  wide 3
│   │   ├──  so wide 1
│   │   ├──  too wide 1
│   │   └──  very wide 1
│   ├──  thick 3
│   │   ├──  so thick 1
│   │   ├──  very thick 1
│   │   └──  too thick 1
│   ├──  short 4
│   │   ├──  too short 1
│   │   ├──  so short 1
│   │   ├──  very short 1
│   │   └──  so shorter 1
│   ├──  fit 8
│   │   ├──  correspond 1
│   │   ├──  conform to 1
│   │   ├──  outfit 1
│   │   ├──  fit out 1
│   │   ├──  match 1
│   │   ├──  suit 1
│   │   ├──  accommodate 1
│   │   └──  set 1
│   ├──  big 11
│   │   ├──  too spaced 1
│   │   ├──  so big 1
│   │   ├──  very large 1
│   │   ├──  too large 1
│   │   ├──  too big 1
│   │   ├──  so huge 1
│   │   ├──  so bulky 1
│   │   ├──  very big 1
│   │   ├──  so large 1
│   │   ├──  too baggy 1
│   │   └──  too bulky 1
│   ├──  

'poor product'

# Original Complain Factor

In [38]:
ori_root  = Node('Zinus Original Complain Factor')

In [39]:
# Add Nodes
for idx1, fc1 in enumerate( org_df['cmpl_fc1'].drop_duplicates() ) :
    # Complain Factor 1 Level
    if (fc1 is None) & (fc1 in ['None','none']):
        pass
    else:
        org_fc1_node = Node(fc1, parent=ori_root)
    for idx2, fc2 in enumerate( set(org_df[org_df['cmpl_fc1']==fc1]['cmpl_fc2']) ) :
        # Complain Factor 2 Level
        if (fc2 is None) & (fc2 in ['None','none']):
            pass
        else:
            org_fc2_node = Node(fc2, parent=org_fc1_node)
        
                

In [42]:
# Rendering with leaves count
for pre, fill, node in RenderTree(ori_root):
    print("%s %s %d" % (pre, node.name, len(node.leaves)))
    
node.name

 Zinus Original Complain Factor 148
├──  Overall quality 1
│   └──  None 1
├──  Odor 5
│   ├──  Odor 1
│   ├──  Mildew 1
│   ├──  Mold 1
│   ├──  Chemical Smell 1
│   └──  Smell 1
├──  Defective 21
│   ├──  Slat 1
│   ├──  Leg 1
│   ├──  Mold 1
│   ├──  Screw hole 1
│   ├──  Weld 1
│   ├──  Bug 1
│   ├──  Finish 1
│   ├──  Mislabeled 1
│   ├──  Middle Support Bar 1
│   ├──  tighten 1
│   ├──  Unstable 1
│   ├──  alignment 1
│   ├──  Broken 1
│   ├──  Sticker 1
│   ├──  Bedbug 1
│   ├──  Glue 1
│   ├──  Stain 1
│   ├──  Rust 1
│   ├──  scratched 1
│   ├──  Wrong part 1
│   └──  huge hole 1
├──  Uncomfortable 2
│   ├──  Clip 1
│   └──  Gap between slat 1
├──  Too soft 2
│   ├──  too soft 1
│   └──  too firm 1
├──  Durability 20
│   ├──  Slat 1
│   ├──  Leg 1
│   ├──  Firmness 1
│   ├──  Weld 1
│   ├──  Spring 1
│   ├──  Clip 1
│   ├──  Depression 1
│   ├──  Velcro 1
│   ├──  Unstable 1
│   ├──  Loose Screw 1
│   ├──  Broken 1
│   ├──  sinking 1
│   ├──  Tear 1
│   ├──  Glue 1
│   ├──  Pl

'Unstable'

# ---------------------
# TEST

In [6]:
comp = df.filter(regex='fc',axis=1)
comp = comp[comp.columns.drop(list(df.filter(regex='cd',axis=1)))]

tree_depth = len(comp.columns)
check = 0
while check < tree_depth :
    if check == 0: 
        name1 = 'cmpl_fc'+str(check+1)
    
        node = Node(df[name1].drop_duplicates().tolist(),parent=root)
    elif check < tree_depth:
        name1 = 'cmpl_fc'+str(check )
        name2 = 'cmpl_fc'+str(check+1) 
        node = Node((set(df[df[name1].isin(df[name1].drop_duplicates().tolist())][name2])), parent=node)
    elif check == tree_depth:
        name1 = 'cmpl_fc'+str(check-1)
        name2 = 'cmpl_fc'+str(check)
        node = Node(set(df[(df[name1].isin(df[name1].drop_duplicates().tolist())) & (df[name2].isin(df[name2].drop_duplicates().tolist()))]['synonym']),parent=nonde)
    check = check +1

cmpl_fc1


In [13]:
root_test = Node('Complain Factor')

CF001 = Node('durability', parent=root_test)
CF002 = Node('defective', parent=root_test)

CF001_001 = Node('slat', parent=CF001)
CF001_002 = Node('clip', parent=CF001)
CF001_003 = Node('tear', parent=CF001)
CF002_001 = Node('stain', parent=CF002)
CF002_002 = Node('bug', parent=CF002)
CF002_003 = Node('finish', parent=CF002)

CF001_001_001 = Node('spline', parent=CF001_001)
CF001_002_001 = Node('clipping', parent=CF001_002)
CF001_003_001 = Node('rip', parent=CF001_003)
CF001_003_001 = Node('bust', parent=CF001_003)
CF001_003_001 = Node('rupture', parent=CF001_003)
CF002_001_001 = Node('discoloration', parent=CF002_001)
CF002_001_001 = Node('dirt', parent=CF002_001)
CF002_001_001 = Node('filth', parent=CF002_001)
CF002_002_001 = Node('hemipterous', parent=CF002_002)
CF002_002_001 = Node('insect', parent=CF002_002)
CF002_002_001 = Node('cimex', parent=CF002_002)
CF002_003_001 = Node('coating', parent=CF002_003)
CF002_003_001 = Node('finishing', parent=CF002_003)
CF002_003_001 = Node('refinement', parent=CF002_003)


for pre, fill, node in RenderTree(root_test):
    print("%s %s" % (pre, node.name))

 Complain Factor
├──  durability
│   ├──  slat
│   │   └──  spline
│   ├──  clip
│   │   └──  clipping
│   └──  tear
│       ├──  rip
│       ├──  bust
│       └──  rupture
└──  defective
    ├──  stain
    │   ├──  discoloration
    │   ├──  dirt
    │   └──  filth
    ├──  bug
    │   ├──  hemipterous
    │   ├──  insect
    │   └──  cimex
    └──  finish
        ├──  coating
        ├──  finishing
        └──  refinement


In [10]:
comp = df.filter(regex='fc',axis=1)
comp = comp[comp.columns.drop(list(df.filter(regex='cd',axis=1)))]

tree_depth = len(comp.columns)
check = 0
while check < tree_depth :
    if check == 0: 
        name1 = 'cmpl_fc'+str(check+1)
        print(name1)
        node = Node(df[name1].drop_duplicates().tolist(),parent=root)
    elif check < tree_depth:
        name1 = 'cmpl_fc'+str(check )
        name2 = 'cmpl_fc'+str(check+1) 
        node = Node((set(df[df[name1].isin(df[name1].drop_duplicates().tolist())][name2])), parent=node)
    elif check == tree_depth:
        name1 = 'cmpl_fc'+str(check-1)
        name2 = 'cmpl_fc'+str(check)
        node = Node(set(df[(df[name1].isin(df[name1].drop_duplicates().tolist())) & (df[name2].isin(df[name2].drop_duplicates().tolist()))]['synonym']),parent=nonde)
    check = check +1

cmpl_fc1
